In [ ]:
%pip install -U insightface onnxruntime-gpu  numpy fastai pillow

In [ ]:
%pip install -U gdown moviepy ffmpeg insightface natsort hdbscan imutils  seaborn annoy

In [ ]:
!sudo apt install ffmpeg

In [ ]:

!sudo apt update
!sudo apt install streamlink -y

In [ ]:
import gdown
import cv2
import os
from moviepy.editor import VideoFileClip,AudioFileClip
import glob
import os.path as osp
import shutil
import insightface
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image
import seaborn as sns
import pandas as pd
import pickle
from imutils import build_montages
from sklearn.cluster import DBSCAN
from hdbscan import HDBSCAN
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox



In [ ]:
# Download video from Google Drive
url = f'https://drive.google.com/uc?id=18wEUfMNohBJ4K3Ly5wpTejPfDzp-8fI8'
gdown.download(url, "antilop.zip", quiet=False)
!unzip antilop.zip -d antilop

In [ ]:

# Define variables
drive_file_id = '1kgIl4rCSaVQ_PLrMfa4AJQPo5YuKFwFO'  # replace 'FILE_ID' with your file's ID
video_output = 'my_video.mp4'
frame_folder = '/notebooks/frame_folder'
swapped_folder = '/notebooks/swapped'
character_folder = '/notebooks/character'


In [ ]:
shutil.rmtree(frame_folder)
shutil.rmtree(swapped_folder)

In [ ]:
assert insightface.__version__ >= '0.7'


# Create the frame folder if it doesn't exist
if not os.path.exists(frame_folder):
    os.makedirs(frame_folder)
# Create the frame folder if it doesn't exist
if not os.path.exists(swapped_folder):
    os.makedirs(swapped_folder)
# Your face swapping script
app = FaceAnalysis(model='./antilop')
app.prepare(ctx_id=0, det_size=(640, 640))
swapper = insightface.model_zoo.get_model('/notebooks/inswapper_128.onnx')

In [ ]:


import numpy as np

def get_first_face(image):
    # If the image is a string (presumably a file path), read the image
    if isinstance(image, str):
        image = cv2.imread(image)

    # Check if the image is a valid numpy array
    if isinstance(image, np.ndarray):
        source_faces = app.get(image)
        source_faces = sorted(source_faces, key=lambda x: x.bbox[0])
        if len(source_faces) == 0:
            print(image)
            assert False

        return source_faces[0]
    print(image)
    assert False


In [ ]:
import requests

url = "http://example.com/path_to_your_file"  # replace with your file's URL
response = requests.get(url)

with open(video_output, 'wb') as f:  # replace with the path where you want to save the file
    f.write(response.content)

In [ ]:
import gdown
# Download video from Google Drive
url = f'https://drive.google.com/uc?id={drive_file_id}'
gdown.download(url, video_output, quiet=False)

In [ ]:
%pip install  streamlink

In [ ]:
cmd = f'''-o {video_output} "https://www.youtube.com/watch?v=_q5vw2MupE0" 360p'''
!python -m streamlink {cmd}

In [ ]:
# Delete if swapped folder is not empty
if  os.path.exists(character_folder):
    shutil.rmtree(character_folder)

In [ ]:
import cv2
import numpy as np
import insightface
from insightface.app import FaceAnalysis
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
import pickle
import datetime
# Create the frame folder if it doesn't exist
if not os.path.exists(character_folder):
    os.makedirs(character_folder)
if not os.path.exists(frame_folder):
    os.makedirs(frame_folder)
# Initialize the FaceAnalysis application
""" app = FaceAnalysis(providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
app.prepare(ctx_id=0, det_size=(640, 640))  # Use GPU device 0 and input image size as (640, 640) """

# Open the video file
cap = cv2.VideoCapture(video_output)
# Calculate frame rate (frames per second)
fps = cap.get(cv2.CAP_PROP_FPS)
face_index= 0
# Initialize a list to store embeddings and face data for each detected face
face_data = []
count = 0
# Loop through the video file frame by frame
while True:
    ret, frame = cap.read()
    if not ret:
        break
        # Calculate the time in minutes and seconds
    time_in_milliseconds = cap.get(cv2.CAP_PROP_POS_MSEC)
    frame_time = datetime.timedelta(milliseconds=time_in_milliseconds)
    path = os.path.join(frame_folder, f"{count}.jpg")
    cv2.imwrite(path, frame)  # save frame as JPEG file
    # Use the FaceAnalysis application to detect faces in the frame
    faces = app.get(frame)

    # For each detected face, extract the embedding, bounding box, and face image, and add them to the list
    for face in faces:
        face_embedding = face.embedding
        bbox = face.bbox.astype(int)
        cropped_face = frame[bbox[1]:bbox[3], bbox[0]:bbox[2]]
        if cropped_face.size > 0:
            face_data.append({'index':face_index,'label':'','frame':count,'time':frame_time,'face_embedding': face_embedding,'score':face.det_score,'normed_embedding': face.normed_embedding, 'bbox': bbox, 'image': cv2.cvtColor(cropped_face, cv2.COLOR_BGR2RGB)})
        face_index += 1
    
    count += 1

# Save the face_data to disk
with open('face_data.pkl', 'wb') as f:
    pickle.dump(face_data, f)


In [ ]:
import pickle
class FaceDataManager:
    def __init__(self, file_path):
        self.file_path = file_path
        self.load_data()

    def load_data(self):
        with open(self.file_path, 'rb') as f:
            self.data = pickle.load(f)

    def save_data(self):
        with open(self.file_path, 'wb') as f:
            pickle.dump(self.data, f)

    def update_label(self, frame_number, face_image, new_label,save= True):
        for f in self.data:
            if f['frame'] == frame_number and np.array_equal(f['image'], face_image):
                f['label'] = new_label
        if save:
            self.save_data()
    def update_labels(self,old_label ,new_label):
        for f in self.data:
            if f['label'] == old_label:
                f['label'] = new_label
        self.save_data()
    
global face_data_manager
face_data_manager = FaceDataManager('face_data.pkl')
labels = [f['label'] for f in face_data_manager.data]
# get unique labels
""" labels = list(set(labels)) """

In [ ]:
# Load the data
face_data_manager.load_data()

data = []
images = []
video = []
chunk = []
prev_time = None

for face in face_data_manager.data:
    current_time = face['time']

    # If it's the first face or at least one second has passed, process the face
    # if prev_time is None or (current_time - prev_time).total_seconds() >= 0.5:
    data.append(face['normed_embedding'])
    images.append(face['image'])
    video.append("output.mp4")
    chunk.append(1)
    prev_time = current_time

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

def scatter_thumbnails(data, images, zoom=0.12, colors=None):
    assert len(data) == len(images)

    # reduce embedding dimensions to 2
    x = PCA(n_components=2).fit_transform(data) if len(data[0]) > 2 else data

    # create a scatter plot.
    f = plt.figure(figsize=(22, 15))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], s=4)
    _ = ax.axis('off')
    _ = ax.axis('tight')

    # add thumbnails :)
    for i in range(len(images)):
        image = images[i]
        im = OffsetImage(image, zoom=zoom)
        bboxprops = dict(edgecolor=colors[i]) if colors is not None else None
        ab = AnnotationBbox(im, x[i], xycoords='data',
                            frameon=(bboxprops is not None),
                            pad=0.02,
                            bboxprops=bboxprops)
        ax.add_artist(ab)
    return ax


In [ ]:
import cv2
import numpy as np
import insightface
from insightface.app import FaceAnalysis
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
import pickle
import datetime
import os



# Call the scatter_thumbnails function
scatter_thumbnails(data, images)
plt.title('Facial Embeddings - Principal Component Analysis')
plt.show()


In [ ]:
x= data

In [ ]:
from sklearn.manifold import TSNE
# PCA first to speed it up
x = PCA(n_components=4).fit_transform(data)
x = TSNE(perplexity=9,
         n_components=3).fit_transform(x)

In [ ]:
_ = scatter_thumbnails(x, images, zoom=0.25)
plt.title('3D t-Distributed Stochastic Neighbor Embedding')
plt.show()

In [ ]:
# !pip install -q hdbscan
import hdbscan
import sklearn.cluster as cluster
from sklearn.cluster import MeanShift
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import OPTICS
from numpy.linalg import norm

def calculate_cosine_distance(feat1, feat2):
    feat1 = feat1.ravel()
    feat2 = feat2.ravel()
    sim = np.dot(feat1, feat2) / (norm(feat1) * norm(feat2))
    return 1 - sim
def plot_clusters(data, algorithm, *args, **kwds):
    cluster = algorithm(*args, **kwds)
    labels = algorithm(*args, **kwds).fit_predict(data)
    palette = sns.color_palette('deep', np.max(labels) + 1)
    colors = [palette[x] if x >= 0 else (0,0,0) for x in labels]
    # ax = scatter_thumbnails(x, images, 0.06, colors)
    plt.title(f'Clusters found by {algorithm.__name__}')
    return cluster ,labels

""" clustering,labels = plot_clusters(x, hdbscan.HDBSCAN, alpha=1.0, min_cluster_size=10, min_samples=70)  """
""" clusters = plot_clusters(x, cluster.DBSCAN, n_jobs=-1, eps=0.52, min_samples=1) """
""" clusters = plot_clusters(x, AffinityPropagation) """
""" clustering = plot_clusters(x, OPTICS, min_samples=120)  """
""" clusters = plot_clusters(x, MeanShift) """

clustering,labels = plot_clusters(x, hdbscan.HDBSCAN, alpha=0.82, min_cluster_size=10, min_samples=120,metric=calculate_cosine_distance)  
for i,label in enumerate(labels):
    face_data_manager.data[i]["label"]= str(label.item())
face_data_manager.save_data()
list(set(labels))

In [ ]:
from annoy import AnnoyIndex

df= pd.DataFrame(face_data_manager.data)
X= x
f = len(X[0])
t = AnnoyIndex(f, metric="dot")
ntree = 8 

for i, vector in enumerate(X):
    t.add_item(i, vector)
_  = t.build(ntree)
def get_similar_images_annoy(img_index, n=8, max_dist=0.83):
    vid, face  = df.iloc[img_index, [0,8]]
    similar_img_ids, dist = t.get_nns_by_item(img_index, n+1, include_distances=True)
    similar_img_ids = [s for s,d in zip(similar_img_ids, dist) if d <= max_dist][1:]  # first item is always its own video
    return vid, face, df.iloc[similar_img_ids], dist
def get_sample_n_similar(sample_idx):
    vid, face, similar, distances = get_similar_images_annoy(sample_idx)
    
    fig = plt.figure(figsize=(15, 7))
    gs = fig.add_gridspec(2, 6)
    ax1 = fig.add_subplot(gs[0:2, 0:2])
    ax2 = fig.add_subplot(gs[0, 2])
    ax3 = fig.add_subplot(gs[0, 3])
    ax4 = fig.add_subplot(gs[0, 4])
    ax5 = fig.add_subplot(gs[0, 5])
    ax6 = fig.add_subplot(gs[1, 2])
    ax7 = fig.add_subplot(gs[1, 3])
    ax8 = fig.add_subplot(gs[1, 4])
    ax9 = fig.add_subplot(gs[1, 5])
    axx = [ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9]
    for ax in axx:
        ax.set_axis_off()
    list_plot = [face] + similar['image'].values.tolist()
    list_cluster = [df.iloc[sample_idx]['label']] + similar['label'].values.tolist()
    for ax, face, cluster, dist in zip(axx, list_plot, list_cluster, distances):
        ax.imshow(face)
        ax.set_title(f' @{dist:.2f}\label:{cluster}') # show video filename and distance
# display samples and their nearest neighbors
for i in np.random.choice(len(X), 10, replace=False):
    get_sample_n_similar(i)


In [ ]:
# Dictionary to hold mappings of old cluster to new cluster
merge_map = {}
df = pd.DataFrame(face_data_manager.data)
# Iterate over all clusters
for cluster in df['label'].unique():
    # Get representative item for this cluster (for simplicity, we choose the first one)
    representative_item = df[df['label'] == cluster].index[0]

    # Get the closest clusters using Annoy
    similar_items, distances = t.get_nns_by_item(representative_item, n=9, include_distances=True)

    # Check the second closest item's distance (the closest will be the item itself)
    for dist in distances:
        if dist <= 0.28:
            similar_item = similar_items[1]
            similar_cluster = df.loc[similar_item, 'label']
            merge_map[cluster] = similar_cluster
    else:
        # Otherwise, this cluster stays independent
        merge_map[cluster] = cluster

# Now update the 'cluster' field using the merge_map
df = pd.DataFrame(face_data_manager.data)
print(df["label"].nunique())
for i,data in enumerate(face_data_manager.data):
    face_data_manager.data[i]["label"]= merge_map[data["label"]]
face_data_manager.save_data()
df = pd.DataFrame(face_data_manager.data)

df["label"].nunique()

In [ ]:

# Delete if swapped folder is not empty
if  os.path.exists(character_folder):
    shutil.rmtree(character_folder)

In [ ]:

for label in list(set(labels)):
    # Skip the noise
    if label == -1:
        continue  

    directory_path = os.path.join(character_folder, str(label))
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

    # Select samples associated with the current label
    idxs = np.where(labels == label)[0]
    faces = []
    # loop over the sampled indexes
    for i in idxs:
        face_data_manager.data[i]['label'] = str(label)
    idxs = np.random.choice(idxs, size=min(25, len(idxs)), replace=False)
    for i in idxs:
        # Get the time elapsed
        time_elapsed = face_data_manager.data[i]['time']
        minutes = time_elapsed.total_seconds() // 60
        seconds = time_elapsed.total_seconds() % 60
        # Calculate minutes and seconds
        minutes = time_elapsed.total_seconds() // 60
        seconds = time_elapsed.total_seconds() % 60
        # Get the face from the data
        face = face_data_manager.data[i]["image"]
        # Format the time string as MM:SS
        frame_time = "{:02}:{:02}".format(int(minutes), int(seconds))
        # Force resize the face to 96x96 and then add it to the
        # faces montage list
        face = cv2.resize(face, (150, 150))
        cv2.putText(face,frame_time, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        faces.append(face)
    montage = build_montages(faces, (150, 150), (5, 5))[0]
    # Save the output montage
    title = "Face ID #{}".format(label)
    title = "Unknown Faces" if label == -1 else title
    cv2.imwrite(os.path.join(directory_path, title+'.jpg'), montage)
    
face_data_manager.save_data()     

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from fastai.vision.all import *
from pathlib import Path
from PIL import Image as PilImage
import shutil

path = Path(character_folder)
labels = [data["label"] for data in face_data_manager.data if data["label"]]

def rename_folder(old_name, new_name):
    if new_name.exists():  # If the target directory exists
     
        shutil.rmtree(old_name)
    else:
        old_name.rename(new_name)
        
def create_on_button_clicked_handler(label, name):
    global face_data_manager
    def on_button_clicked(b):
        global face_data_manager
        new_label = name.value
        print(f'Character name confirmed: {new_label}')

        face_data_manager.update_labels(label, new_label)

        old_name = Path(Path(path) / label)
        new_name = Path(Path(path) / new_label)

        rename_folder(old_name, new_name)

        clear_output(wait=True)
        next(process)

    return on_button_clicked
def process_folders(labels, path):
    global face_data_manager
    for label in labels:
        image_paths = list((path / label).glob('*.jpg'))
        if not image_paths:
            continue
        print(f"Current cluster: {label}")

        image_resolution = {}
        for image_path in image_paths:
            with PilImage.open(image_path) as img:
                width, height = img.size
                resolution = width * height  # resolution is defined as width * height
                image_resolution[image_path] = resolution

        sorted_image_paths = sorted(image_resolution, key=image_resolution.get, reverse=True)
        name = widgets.Text(value=label, placeholder='Enter character name', description='Name:')
        button = widgets.Button(description='Confirm')
        display(name, button)
        with open(sorted_image_paths[0], "rb") as file:
            image = file.read()
            img_widget = widgets.Image(value=image, format='png', width=1000, height=400)
            display(img_widget)

     


        button.on_click(create_on_button_clicked_handler(label, name))
  
        yield

process = process_folders(labels, path)
next(process)

In [ ]:
for data in face_data_manager.data:
    if data["label"].isnumeric():
        print(data["index"],"is numeric")


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import insightface
from insightface.app import FaceAnalysis
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer

""" data = [] """
""" for label in os.listdir(character_folder):
    for filename in os.listdir(os.path.join(character_folder, label)):
        if filename.endswith('.npy'):  # check if the file is an image
            # read the image
            img_path = os.path.join(character_folder, label, filename)
            embedding = np.load(img_path)
            data.append({'embedding': embedding, 'label': label})
                
 """
# create a dataframe
df = pd.DataFrame(face_data_manager.data)

# Separate features and target
X = x # Convert list of embeddings back to numpy array
y = df["label"]

# Encode labels
labelencoder = LabelEncoder()
y_encoded = labelencoder.fit_transform(y)
# Normalize the embeddings
normalizer = Normalizer(norm='l2')
X = normalizer.transform(X)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2)

# Define the model
knn = KNeighborsClassifier(n_neighbors=9,metric="euclidean")

# Train the model
knn.fit(X_train, y_train)

# Validate the model
accuracy = knn.score(X_val, y_val)
print(f'Validation accuracy: {accuracy}')
def retrain_knn():
    global knn
    global y
    X = x # Convert list of embeddings back to numpy array
    y = df["label"]

    # Encode labels
    labelencoder = LabelEncoder()
    y_encoded = labelencoder.fit_transform(y)
    # Normalize the embeddings
    normalizer = Normalizer(norm='l2')
    X = normalizer.transform(X)

    # Split data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2)

    # Define the model
    knn = KNeighborsClassifier(n_neighbors=9,metric="euclidean")

    # Train the model
    knn.fit(X_train, y_train)

# def classify_face(embedding,face_index = None):
#     """     normalizer = Normalizer(norm='l2')
#     embedding = normalizer.transform([embedding]) """
#     embedding = [embedding]
#     predicted_label = knn.predict(embedding)
#     predicted_label_str = labelencoder.inverse_transform(predicted_label)[0]
#     if predicted_label_str == "-1":
#         annoy_similar_indices, dist = t.get_nns_by_vector(embedding[0], 9, include_distances=True)
#         # Check if the similarity based on Annoy index is above the threshold
#         max_dist = 1
#         if dist[1] <= max_dist:
#             return labels[annoy_similar_indices[1]]
#         return "Unknown"
#     # Additional similarity check
#     """ imilarity_threshold = 0.34
#         predicted_class_embeddings = X_train[y_train == predicted_label[0]]
#         sims_to_predicted_class = np.dot(predicted_class_embeddings, embedding[0])
#         if np.mean(sims_to_predicted_class) < similarity_threshold:
#             annoy_similar_indices, dist = t.get_nns_by_vector(embedding[0], 2, include_distances=True)
#             # Check if the similarity based on Annoy index is above the threshold
#             max_dist = 1
#             if dist[1] <= max_dist:
#                 embedding = x[annoy_similar_indices[1]]
#                 return classify_face(embedding[0],face_index)
#             return "Unknown"
#     """
#     return predicted_label_str


In [ ]:
def classify_face(embedding,face_index = None):
    global face_data_manager
    # normalizer = Normalizer(norm='l2')
    # embedding = normalizer.transform([embedding])
    annoy_similar_indices, dist = t.get_nns_by_vector(embedding, 10, include_distances=True)
    predicted_label_str = face_data_manager.data[annoy_similar_indices[0]]["label"]
    if predicted_label_str == "-1":
        for i in range(1,len(annoy_similar_indices)):
            if face_data_manager.data[annoy_similar_indices[i]]["label"] != "-1":
                return face_data_manager.data[annoy_similar_indices[i]]["label"] 

    # Additional similarity check
    similarity_threshold = 0.28
    predicted_class_embeddings = X_train[y_train == predicted_label_str]
    sims_to_predicted_class = np.dot(predicted_class_embeddings, embedding[0])
    if np.mean(sims_to_predicted_class) < similarity_threshold:
        return "-1"

    return predicted_label_str

In [ ]:
if not os.path.exists(frame_folder):
    os.makedirs(frame_folder)
# Extract frames from video
vidcap = cv2.VideoCapture(video_output)
success, image = vidcap.read()
count = 0
target_imgs = []  # a list of file paths to images of the target faces
while success:
    path = os.path.join(frame_folder, f"{count}.jpg")
    target_imgs.append(path)
    cv2.imwrite(path, image)  # save frame as JPEG file
    success, image = vidcap.read()
    count += 1

In [ ]:
# Delete if swapped folder is not empty
if  os.path.exists(swapped_folder):
    shutil.rmtree(swapped_folder)

In [ ]:
# Create the frame folder if it doesn't exist
if not os.path.exists(swapped_folder):
    os.makedirs(swapped_folder)

In [ ]:
import collections
import random
import natsort
import cv2
from collections import Counter, deque
import matplotlib.pyplot as plt
from imutils import build_montages
global swappings
swappings ={
   'm':get_first_face('/notebooks/swapping/m.jpg'),
   'g':get_first_face('/notebooks/swapping/g.jpg'),
   'la':get_first_face('/notebooks/swapping/la.jpg'),
   'l':get_first_face('/notebooks/swapping/l.jpg'),
   'je':get_first_face('/notebooks/swapping/je.jpg'),
   'b':get_first_face('/notebooks/swapping/b.jpg'),
   'j':get_first_face('/notebooks/swapping/j.jpg'),
   'yb':get_first_face('/notebooks/swapping/yb.jpg'),
   'yb':get_first_face('/notebooks/swapping/yb.jpg'),
   'yl':get_first_face('/notebooks/swapping/yl.jpg'),
}
targets={
"-1":None
}
video = cv2.VideoCapture(video_output)
fps = video.get(cv2.CAP_PROP_FPS)
smoothing_window_size = 5
face_classification_history = collections.defaultdict(lambda: deque(maxlen=int(fps)+1))

start_frame = None
end_frame = None
frame_count = -1
DEBUG = False
results = []  # List to hold face images for montage
face_index = 0


In [ ]:

""" source_face = get_first_face('/notebooks/DSC06729.JPG') """
# Define your file path
import json


attribution_path = "attribution.json"
# Check if the file exists
if os.path.exists(attribution_path):
    # Load the attribution data from the file
    with open(attribution_path, 'r') as f:
        attribution = json.load(f)
else:
    # Create a new empty attribution dictionary
    attribution = {}
unused_keys = list(swappings.keys())
for key in list(targets.keys()):
    value = targets[key]
    if value is not None:
        unused_keys.remove(value)



In [ ]:
while True:
    ret, frame = video.read()
    if not ret:
        break  
    frame_count += 1
    if start_frame is not None and frame_count < start_frame:
        continue
    if end_frame is not None and frame_count > end_frame:
        break

    faces = app.get(frame)
    """     faces = sorted(faces, key=lambda x: x.bbox[0]) """
    res = frame.copy()

    used_indices = set()  # Keep track of used indices in this frame
    face_info = []  # List to store face info (index and position)
    for face in faces:
        index = classify_face(face.normed_embedding,face_index=face_index)
        # if index in used_indices:
        #     continue
        used_indices.add(index)  # Mark this index as used
        """         face_classification_history[index].append(index)
                counter = Counter(face_classification_history[index])
                most_common_index = counter.most_common(1)[0][0] """
        most_common_index = index
        found = False
        bbox = face.bbox.astype(int)
        face_info.append((index,bbox))
        # Store index and position for later use
        """         cropped_face = frame[bbox[1]:bbox[3], bbox[0]:bbox[2]]
                if cropped_face.size < 0:
                    continue """
        """         if face.det_score < 0.5:
                    continue """
        if most_common_index in targets:
            found = True
            value = targets[most_common_index]
            if value is not None:
                res = swapper.get(res, face, swappings[value], paste_back=True)
        if not found:
            if most_common_index in attribution:
                found = True
                value = swappings[attribution[most_common_index]]
                if value is not None:
                    res = swapper.get(res, face, swappings[attribution[most_common_index]], paste_back=True)
            else:
                print("index not found :" ,most_common_index)
        if not found and index != "Unknown":
            if len(unused_keys) == 0:  
                unused_keys = list(swappings.keys())
                for key in list(targets.keys()):
                    if key in unused_keys:
                        unused_keys.remove(key)
            random_key = random.choice(unused_keys)
            unused_keys.remove(random_key)
            print("index ",most_common_index," attribute to ",random_key, " frame ", frame_count)
            attribution[most_common_index] = random_key
            random_value = swappings[random_key]
            res = swapper.get(res, face, random_value, paste_back=True)
        face_index += 1
    cv2.imwrite(osp.join(swapped_folder, '{}.jpg'.format(frame_count)), res)
    if DEBUG:
        debug = res.copy()
        for info in face_info:
            index, bbox = info
            new_bbox = bbox 
            cv2.putText(debug, str(index), (int(new_bbox[0]), int(new_bbox[1])), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3, cv2.LINE_AA)
        debug =  cv2.resize(debug,  (400, 400), interpolation = cv2.INTER_AREA)

        cv2.putText(debug,str(frame_count), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(debug,str(len(face_info)), (350, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2, cv2.LINE_AA)
        results.append(debug)
        # Every fps number of frames, create a montage and display it
        if frame_count % int(fps) == 0:
            montage = build_montages(results, (150, 150), (5, 5))[0]
            # Calculate time elapsed in the video
            time_elapsed = frame_count / fps  # time in seconds
            minutes = int(time_elapsed // 60)
            seconds = int(time_elapsed % 60)
            print(f"Current time in video: {minutes:02}:{seconds:02}")
            # Clear previous output and display the image
            """  clear_output(wait=True) """
            plt.figure(figsize=(400,400))
            plt.imshow(cv2.cvtColor(montage, cv2.COLOR_BGR2RGB))
            plt.axis('off')
            plt.show()
            
            results = []  # Clear the faces list

video.release()

In [ ]:
# Save the updated attribution data back to the file
with open(attribution_path, 'w') as f:
    json.dump(attribution, f) 

In [ ]:
import re
from ipywidgets import HBox, VBox
import ipywidgets as widgets
from IPython.display import display, clear_output
from fastai.vision.all import *
from pathlib import Path
from PIL import Image as PilImage
import shutil
from ipywidgets import GridBox

""" attribution = {} """
""" unused_keys = list(swappings.keys()) """

frame_output = widgets.Output()
path = Path(character_folder)
processed_images = glob.glob(os.path.join(frame_folder, '*.jpg'))
processed_images.sort(key=lambda f: int(re.sub('\D', '', os.path.basename(f))))
def calculate_cosine_similarity(feat1, feat2):
    feat1 = feat1.ravel()
    feat2 = feat2.ravel()
    sim = np.dot(feat1, feat2) / (norm(feat1) * norm(feat2))
    return sim
# -------------------------------- Function definitions ------------------------------
def get_class_label_embedding(label):
    # Gather all embeddings with this label
    embeddings = [d['normed_embedding'] for d in face_data_manager.data if d['label'] == str(label)]
    # Calculate the average embedding
    average_embedding = np.mean(embeddings, axis=0)
    return average_embedding

def create_on_button_clicked(face, frame_number, name, path, character_folder):
    def on_button_clicked(b):
        global face_data_manager
        label = name.value
        face_data_manager.update_label(frame_number, face['image'], label)
        process_frame(frame_number)
    return on_button_clicked

def process_frame(frame_number):
    print(f"Frame number: {frame_number}")
    global face_data_manager
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

    frame_path = processed_images[frame_number]
    frame = cv2.imread(str(frame_path))

    frame_widget = convert_frame_to_widget(frame)
    widgets_to_display = [frame_widget]
    if 'swappings' in globals():
        # Generate the image with swapped faces
        faces = app.get(frame)
        swapped_frame = face_swap(frame, faces,frame_number)
        swapped_frame_widget = convert_frame_to_widget(swapped_frame)
        widgets_to_display.append(swapped_frame_widget)

    faces = [p for p in face_data_manager.data if p["frame"] == int(frame_number)]
    face_widgets = process_faces(faces,frame_number)

    display(HBox(widgets_to_display ))
    display(HBox(face_widgets))
    labels = set(face['label'] for face in face_data_manager.data)
    image_boxes = [display_images(label,faces) for label in labels]

    grid = widgets.GridBox(image_boxes, layout=widgets.Layout(grid_template_columns="repeat(4, 250px)"))
    display(grid)

def convert_frame_to_widget(frame):
    frame_pil = PilImage.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    base_width = 500
    w_percent = (base_width / float(frame_pil.size[0]))
    h_size = int((float(frame_pil.size[1]) * float(w_percent)))
    frame_pil = frame_pil.resize((base_width, h_size))

    img_byte_arr = io.BytesIO()
    frame_pil.save(img_byte_arr, format='PNG')
    return widgets.Image(value=img_byte_arr.getvalue(), format='png')

def process_faces(faces, frame_number):
    face_widgets = []
    for face in faces:
        if len(face_widgets) == 0:
            print(f"Time: {face['time']}")
        cropped_face_widget = convert_face_to_widget(face['image'])
        name = widgets.Text(value=str(face['label']), placeholder='Enter character name', description='Name:')
        button = widgets.Button(description='Confirm')
        on_button_clicked = create_on_button_clicked(face, frame_number, name, path, character_folder)
        button.on_click(on_button_clicked)
        face_box = VBox([cropped_face_widget, name, button])
        face_widgets.append(face_box)
    return face_widgets

def convert_face_to_widget(face):
    face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)  # Add this line to convert the color space
    cropped_face_pil = PilImage.fromarray(face)
    cropped_face_pil = cropped_face_pil.resize((200, 200))
    img_byte_arr = io.BytesIO()
    cropped_face_pil.save(img_byte_arr, format='PNG')
    return widgets.Image(value=img_byte_arr.getvalue(), format='jpg')

def calculate_centroid(embeddings):
    return np.mean(embeddings, axis=0)

def display_images(label,frame_faces):
    central_image ,centroid= find_central_images(label)
    central_image_widget = convert_face_to_widget(central_image)

    # Retrieve the average embedding for this label
    label_embedding = centroid

    # For each face in the current frame, calculate the similarity to this label
    similarity_scores_widgets = []
    # for face in frame_faces:
    #     face_embedding = face['normed_embedding']
    #     similarity = calculate_cosine_similarity(face_embedding, label_embedding)
    #     sl = classify_face(face_embedding)
    #     similarity_scores_widgets.append(widgets.Text(value=f"{similarity:.3f}", description=f'Similarity {sl}:', disabled=True))

    return VBox([widgets.Label(f"Label: {label}"), central_image_widget] + similarity_scores_widgets)

def find_central_images(label):
    label_embeddings = [d['normed_embedding'] for d in face_data_manager.data if d['label'] == str(label)]
    centroid = calculate_centroid(label_embeddings)
    distances = [np.linalg.norm(e - centroid) for e in label_embeddings]
    min_index = np.argmin(distances)
    central_image = [d['image'] for d in face_data_manager.data if d['label'] == str(label)][min_index]
    return central_image,centroid

def on_slider_change(change):
    process_frame(change['new'])
    
def save_swapped_image(b):
    frame_number = slider.value  # or the current frame number
    frame_path = processed_images[frame_number]
    frame = cv2.imread(str(frame_path))

    if 'swappings' in globals():
        # Generate the image with swapped faces
        faces = app.get(frame)
        swapped_frame = face_swap(frame, faces)

        # Save the swapped frame
        save_path = os.path.join(swapped_folder, f'{frame_number}.jpg')
        cv2.imwrite(save_path, swapped_frame)
        print(f'Swapped image saved to {save_path}')
def on_next_button_clicked(b):
    label = label_input.value
    current_frame = slider.value
    start_frame, end_frame = range_slider.value
    
    next_frame = None
    for f in face_data_manager.data:
        if f['label'] == label and start_frame <= f['frame'] <= end_frame and f['frame'] > current_frame:
            next_frame = f['frame']
            break

    if next_frame is not None:
        slider.value = next_frame
    else:
        print(f"No more frames with label '{label}' within the selected range from frame {start_frame} to frame {end_frame}")


def on_change_label_button_clicked(b):
    old_label = old_label_field.value
    new_label = new_label_field.value
    start_frame, end_frame = range_slider.value
    for face_data in face_data_manager.data:
        if face_data['label'] == old_label and start_frame <= face_data['frame'] <= end_frame:
            face_data['label'] = new_label
    face_data_manager.save_data()
def face_swap(frame, faces,frame_number):

    global targets
    global unused_keys
    res = frame.copy()
    used_indices = set()  # Keep track of used indices in this frame
    face_info = []  # List to store face info (index and position)
    for i,face in enumerate(faces):
        index = classify_face(face.normed_embedding)
        used_indices.add(index)  # Mark this index as used
        most_common_index = index
        found = False
        bbox = face.bbox.astype(int)
        face_info.append((index,bbox))
        # Store index and position for later use
        if most_common_index in targets:
            found = True
            value = targets[most_common_index]
            if value is not None:
                res = swapper.get(res, face, swappings[value], paste_back=True)
        if not found:
            if most_common_index in attribution:
                found = True
                value = swappings[attribution[most_common_index]]
                if value is not None:
                    res = swapper.get(res, face, swappings[attribution[most_common_index]], paste_back=True)
            # else:
            #     print("index not found :" ,most_common_index)
        if not found and index != "Unknown":
            if len(unused_keys) == 0:  
                unused_keys = list(swappings.keys())
                for key in list(targets.keys()):
                    if key in unused_keys:
                        unused_keys.remove(key)
            random_key = random.choice(unused_keys)
            unused_keys.remove(random_key)
            # print("index ",most_common_index," attribute to ",random_key, " frame ", frame_count)
            attribution[most_common_index] = random_key
            random_value = swappings[random_key]
            res = swapper.get(res, face, random_value, paste_back=True)
        debug = res
        for info in face_info:
            index, bbox = info
            new_bbox = bbox 
            cv2.putText(debug, str(index), (int(new_bbox[0]), int(new_bbox[1])), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3, cv2.LINE_AA)
        debug =  cv2.resize(debug,  (400, 400), interpolation = cv2.INTER_AREA)

        cv2.putText(debug,str(frame_count), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(debug,str(len(face_info)), (350, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2, cv2.LINE_AA)
        results.append(debug)
    return res

# -------------------------------- Execution starts here ------------------------------
cap = cv2.VideoCapture(video_output)
total_frames = int(len(processed_images))


first_frame_with_label_0 = min([f['frame'] for f in face_data_manager.data if f['label'] == '0' or f['label'] == ''], default=0)
fps = cap.get(cv2.CAP_PROP_FPS)

save_button = widgets.Button(description='Save Swapped Image')
save_button.on_click(save_swapped_image)

display(save_button)

slider = widgets.IntSlider(min=0, max=int(total_frames-1), step=fps/4, description='Frame:')
slider.observe(on_slider_change, names='value')

label_input = widgets.Text(placeholder='Enter character label', description='Label:')
next_button = widgets.Button(description='Go to next')
next_button.on_click(on_next_button_clicked)

display(HBox([label_input, next_button]))

old_label_field = widgets.Text(value='', placeholder='Enter old label', description='Old Label:')
new_label_field = widgets.Text(value='', placeholder='Enter new label', description='New Label:')
change_label_button = widgets.Button(description='Change Label')
change_label_button.on_click(on_change_label_button_clicked)

range_slider = widgets.IntRangeSlider(value=[0, total_frames-1], min=0, max=total_frames-1, step=fps, description='Frame Range:',)
input_widgets = widgets.HBox([range_slider, old_label_field, new_label_field, change_label_button])

display(input_widgets)
display(slider)

slider.value = 0
display(frame_output)

In [ ]:
processed_images = glob.glob(os.path.join(swapped_folder, '0*.jpg'))
for file in processed_images:
    os.remove(file)

In [ ]:
import subprocess

output_video = "/notebooks/output.mp4"
video = cv2.VideoCapture(video_output)
fps = video.get(cv2.CAP_PROP_FPS)
video.release()

# Get the processed images
processed_images = glob.glob(os.path.join(swapped_folder, '*.jpg'))

# Sort the processed images (this may be necessary depending on how your files are named)
processed_images.sort()
print(len(processed_images))
# Initialize the video writer
height, width, _ = cv2.imread(processed_images[0]).shape
print(height, width)

# Define the command
command = f'ffmpeg -y -r {fps} -s {width}x{height} -i {swapped_folder}/%01d.jpg -vcodec libx264  -pix_fmt yuv420p {output_video}'

# Execute the command
subprocess.call(command, shell=True)

In [ ]:
from IPython.display import Video

Video(output_video)

In [ ]:
import subprocess
""" from pydub import AudioSegment

def convert_wav_to_aac(wav_file_path, aac_file_path):
    audio = AudioSegment.from_wav(wav_file_path)
    audio.export(aac_file_path, format="mp3") """

# Call the function with your files

""" convert_wav_to_aac('output.wav', 'output.mp3') """
temp_audio_file = 'output.aac'

# Remove temporary audio file if it exists
if os.path.exists(temp_audio_file):
    os.remove(temp_audio_file)

# Remove output video with audio file if it exists
output_with_audio_file = '/notebooks/output_with_audio.mp4'
if os.path.exists(output_with_audio_file):
    os.remove(output_with_audio_file)

# Extract audio from original video and save it as a temporary audio file
audio_extraction_command = f'ffmpeg -y -i {video_output} -vn -acodec aac -strict -2 {temp_audio_file}'
subprocess.run(audio_extraction_command, shell=True)

# Combine swapped video with original audio
video_combination_command = f'ffmpeg -y -i {output_video} -i {temp_audio_file} -c:v copy -c:a copy -map 0:v:0 -map 1:a:0 {output_with_audio_file}'
subprocess.run(video_combination_command, shell=True)

WHISPER

In [ ]:
%pip install --upgrade git+https://github.com/coqui-ai/TTS torch torchvision   pydub tensorflow soundfile openai-whisper librosa git+https://github.com/huggingface/transformers.git speechbrain sentencepiece
%pip install git+https://github.com/espnet/espnet
%pip install -q espnet_model_zoo

In [ ]:
!git clone https://github.com/coqui-ai/TTS
%cd /notebooks/TTS
%pip install -e .

In [ ]:
%cd /notebooks/TTS

In [ ]:
%pip install -e .

In [ ]:
%pip install TTS

In [ ]:

# Example voice cloning with YourTTS in English, French and Portuguese
from TTS.api import TTS

from moviepy.editor import AudioFileClip
import librosa
import soundfile as sf
from IPython.display import display, Audio
from bark import SAMPLE_RATE, generate_audio, preload_models
from scipy.io.wavfile import write as write_wav
from IPython.display import Audio
""" 
tts = TTS(model_name="tts_models/multilingual/multi-dataset/your_tts", progress_bar=False, gpu=True)
tts.tts_to_file("Salut c'est louis en vlog.L'Alter Real est une lignée du cheval Lusitanien considérée comme particulièrement pure, généralement décrite comme une race de chevaux de selle d'origine portugaise à part entière. La sélection de cette lignée commence au xviiie siècle, avec un grand succès grâce à l'implication de la famille royale portugaise et à son usage par des écuyers renommés. L'Alter Real connaît une éclipse durant le siècle suivant, puis est sévèrement menacé de disparition et sauvé par le Dr Ruy d'Andrade. Son élevage est depuis soutenu par le gouvernement portugais. Très proche du Lusitanien avec lequel il est confondu, mais aussi du Pure race espagnole et du Minorquin, l'Alter Real est un excellent cheval de dressage, plusieurs fois représenté aux Jeux olympiques. Il présente très souvent une robe baie. ", speaker_wav="audio.wav", language="fr-fr", file_path="cloning_tts.wav") """

In [ ]:
from TTS.api import TTS

In [ ]:
tts = TTS(model_name="voice_conversion_models/multilingual/vctk/freevc24", progress_bar=False, gpu=True)
tts.voice_conversion_to_file(source_wav="upgrade.wav", target_wav="target-upgrade.wav", file_path="output.wav")



In [ ]:
input_waveform, input_sampling_rate= librosa.load('output.wav', sr=None)
Audio(input_waveform, rate=input_sampling_rate)

In [ ]:
import whisper

model = whisper.load_model("base")

# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("/notebooks/temp_audio.aac")
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
options = whisper.DecodingOptions(without_timestamps=False)
result = whisper.decode(model, mel, options)

# print the recognized text
print(result)

In [ ]:
from moviepy.editor import AudioFileClip
import librosa
import soundfile as sf
def extract_audio_from_video(video_file_path , output_file_path,start_time = None, end_time= None, sr=16000):
    audio = AudioFileClip(video_file_path)
    if  start_time is not  None and end_time is not  None:
        audio = audio.subclip(start_time, end_time)
    audio.write_audiofile(output_file_path)
        # Save the audio with the original sampling rate
    temp_file_path = "temp_audio.wav"
    audio.write_audiofile(temp_file_path)
    
    # Load the audio with the original sampling rate
    signal, sr_orig = librosa.load(temp_file_path, sr=None)
    if len(signal.shape) > 1:
        signal = librosa.to_mono(signal.T)    
    # Resample the audio to 16kHz
    signal_resampled = librosa.resample(signal, orig_sr=sr_orig, target_sr=sr)
    
    # Save the resampled audio
    sf.write(output_file_path, signal_resampled, sr)

# Call the function with your files
extract_audio_from_video('Tamioc, le nouveau podcast du Pacifique.mp4', output_file_path='audio.wav')
extract_audio_from_video(video_output, output_file_path='temp_audio.wav')

In [ ]:
import torch
import torchaudio
import numpy as np
from speechbrain.pretrained import EncoderClassifier
import torch.nn.functional as F
import librosa


def generate_speaker_embedding(audio_file_path, model_name="speechbrain/spkrec-xvect-voxceleb"):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    classifier = EncoderClassifier.from_hparams(source=model_name, run_opts={"device": device})

    # Load the audio file
    signal, fs = torchaudio.load(audio_file_path)

    assert fs == 16000, fs

    # Generate the speaker embedding
    with torch.no_grad():
        embeddings = classifier.encode_batch(signal)
        embeddings = F.normalize(embeddings, dim=2)
        embeddings = embeddings.squeeze().cpu().numpy()

    return embeddings

# Call the function with your audio file
embedding = generate_speaker_embedding('audio.wav')

# Save the embedding to a numpy file
np.save('speaker.npy', embedding)

In [ ]:
import librosa
import numpy as np
import torch
from transformers import SpeechT5Processor, SpeechT5ForSpeechToSpeech, SpeechT5HifiGan
import soundfile as sf
from datasets import load_dataset

checkpoint = "microsoft/speecht5_vc"
processor = SpeechT5Processor.from_pretrained(checkpoint)
model = SpeechT5ForSpeechToSpeech.from_pretrained("microsoft/speecht5_vc")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

def process_audio(sampling_rate, waveform):
    if len(waveform.shape) > 1:
        waveform = librosa.to_mono(waveform.T)

    if sampling_rate != 16000:
        waveform = librosa.resample(waveform, orig_sr=sampling_rate, target_sr=16000)

    waveform = waveform
    waveform = torch.tensor(waveform)
    return waveform

def voice_conversion(input_audio_path, speaker_audio_path, output_audio_path):
    # Load and process the input audio
    input_waveform, input_sampling_rate= librosa.load(input_audio_path, sr=None)
    input_waveform = process_audio(input_sampling_rate, input_waveform)

    # Load the speaker embedding
    speaker_embedding = np.load(speaker_audio_path)
    speaker_embedding = torch.tensor(speaker_embedding).unsqueeze(0)  # Remove the first dimension
    inputs = processor(audio=input_waveform, sampling_rate=16000, return_tensors="pt")
    
    # Perform the voice conversion
    speech = model.generate_speech(inputs["input_values"], speaker_embeddings = speaker_embedding, vocoder=vocoder)

    
    # Write the output audio to a file
    sf.write(output_audio_path, speech, 16000)

dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
dataset = dataset.sort("id")
example = dataset[40]
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
np.save("test.npy",torch.tensor(embeddings_dataset[7306]["xvector"]))
sf.write("test.wav",example["audio"]["array"], 16000)

# Call the function with your files
voice_conversion('upgrade.wav', 'bark_voices/louis/speaker.npz', 'output_audio.wav')


In [ ]:
input_waveform, input_sampling_rate= librosa.load('output.wav', sr=None)
Audio(input_waveform, rate=input_sampling_rate)

In [ ]:
from IPython.display import Audio
sampling_rate = dataset.features["audio"].sampling_rate
inputs = processor(audio=example["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)
speech = model.generate_speech(inputs["input_values"], speaker_embeddings, vocoder=vocoder)
Audio(speech, rate=16000)

In [ ]:
import os

import soundfile
from IPython.display import display, Audio
from TTS.api import TTS
mixwav_mc, sr = soundfile.read("audio.wav")
# mixwav.shape: num_samples, num_channels
mixwav_sc = mixwav_mc
display(Audio(mixwav_mc, rate=sr))

In [ ]:
from espnet_model_zoo.downloader import ModelDownloader

d = ModelDownloader()
cfg = d.download_and_unpack("espnet/Wangyou_Zhang_chime4_enh_train_enh_conv_tasnet_raw")

In [ ]:
# Load the model
# If you encounter error "No module named 'espnet2'", please re-run the 1st Cell. This might be a colab bug.
import sys
import soundfile
from espnet2.bin.enh_inference import SeparateSpeech


separate_speech = {}
# For models downloaded from GoogleDrive, you can use the following script:
enh_model_sc = SeparateSpeech(
  train_config=cfg["train_config"],
  model_file=cfg["model_file"],
  # for segment-wise process on long speech
  normalize_segment_scale=False,
  show_progressbar=True,
  ref_channel=4,
  normalize_output_wav=True,
  device="cuda:0",
)

In [ ]:

import soundfile as soundfile

mixwav_mc, sr = soundfile.read("temp_audio.wav")
# mixwav.shape: num_samples, num_channels
mixwav_sc = mixwav_mc
wave = enh_model_sc(mixwav_sc[None, ...], sr)
sf.write("upgrade.wav",wave[0].squeeze(), sr)
display(Audio(wave[0].squeeze(), rate=sr))


mixwav_mc, sr = soundfile.read("bark_voices/gael/speaker.wav")
""" if sr != 16000:
    mixwav_mc = librosa.resample(mixwav_mc, orig_sr=sr, target_sr=16000)
if len(mixwav_mc.shape) > 1:
    mixwav_mc = librosa.to_mono(mixwav_mc.T)  """
# mixwav.shape: num_samples, num_channels
mixwav_sc = mixwav_mc
wave = enh_model_sc(mixwav_sc[None, ...], sr)
sf.write("bark_voices/gael/speaker.wav",wave[0].squeeze(), sr)
display(Audio(wave[0].squeeze(), rate=sr))

""" mixwav_mc, sr = soundfile.read("output.wav")
# mixwav.shape: num_samples, num_channels
mixwav_sc = mixwav_mc
wave = enh_model_sc(mixwav_sc[None, ...], sr)
sf.write("output-upgrade.wav",wave[0].squeeze(), sr)
display(Audio(wave[0].squeeze(), rate=sr)) """


In [ ]:
text = """
Dans un village charmant au bord de la mer, vivait une famille extraordinaire.
"""
from TTS.tts.configs.bark_config import BarkConfig
from TTS.tts.models.bark import Bark

config = BarkConfig()
model = Bark.init_from_config(config)
model.load_checkpoint(config, checkpoint_dir="/root/.local/share/tts/tts_models--multilingual--multi-dataset--bark", eval=True)

""" # with random speaker
output_dict = model.synthesize(text, config, speaker_id="random", voice_dirs=None)
 """
# cloning a speaker.
# It assumes that you have a speaker file in `bark_voices/speaker_n/speaker.wav` or `bark_voices/speaker_n/speaker.npz`
output_dict = model.synthesize(text, config, speaker_id="gael", voice_dirs="bark_voices/")

In [ ]:
from TTS.api import TTS

# Load the model to GPU
# Bark is really slow on CPU, so we recommend using GPU.
tts = TTS("tts_models/multilingual/multi-dataset/bark", gpu=True) 

# Cloning a new speaker
# This expects to find a mp3 or wav file like `bark_voices/new_speaker/speaker.wav`
# It computes the cloning values and stores in `bark_voices/new_speaker/speaker.npz`
""" tts.tts_to_file(text="Hello, how are you?",
                file_path="output.wav",
                voice_dir="bark_voices/") """


# When you run it again it uses the stored values to generate the voice.
tts.tts_to_file(text="""
Dans un village charmant au bord de la mer, vivait une famille extraordinaire. Le père, l'architecte ambitieux, avait une voix grave et rauque. La mère, la pianiste accomplie, parlait avec douceur et finesse. Leur fille unique, une poétesse talentueuse, avait une voix mélodieuse.

Un jour d'été, une lettre mystérieuse est arrivée. "Chère famille, une aventure incroyable vous attend à l'autre bout du monde. Montez à bord du bateau qui quitte le port demain à l'aube, et laissez-vous guider par le vent et les vagues."

Emballés par l'idée, ils ont fait leurs valises. Ils ont rencontré des gens fantastiques lors de leur voyage : un marin à la voix rauque, une conteuse à la voix douce et un enfant qui chantait avec une voix cristalline. Chaque personne qu'ils ont rencontrée avait une histoire à raconter.               
""",
                file_path="output.wav",
                voice_dir="bark_voices",
                speaker="gael") 



In [ ]:
from pydub import AudioSegment
def convert_m4a_to_wav(m4a_filepath, wav_filepath):
    audio = AudioSegment.from_file(m4a_filepath, format="m4a")
    audio = audio.set_frame_rate(SAMPLE_RATE)
    audio = audio.split_to_mono()[0]
    audio.export(wav_filepath, format="wav")
def convert_ogg_to_wav(ogg_file_path, wav_file_path):
    audio = AudioSegment.from_ogg(ogg_file_path)
    audio.export(wav_file_path, format="wav")

# usage
convert_m4a_to_wav("/notebooks/20230715_173011.m4a", "bark_voices/gael/speaker.wav")
mixwav_mc, sr = soundfile.read("bark_voices/gael/speaker.wav")
print(sr)
display(Audio("bark_voices/gael/speaker.wav", rate=sr))


In [ ]:
!tts --list_models

In [ ]:
!rm -rf /root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v1

In [ ]:
!pip install deepfilternet


In [ ]:
!deepFilter louis.ogg -o louis.wav

In [ ]:
from TTS.api import TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v1").to("cuda")


In [ ]:
tts.tts_to_file("C'est juste pour découvrir la room", speaker_wav="test2.wav", language="fr", file_path="output.wav")

In [ ]:
from IPython.display import display, Audio
Audio("output.wav")

In [ ]:
# tts = TTS("tts_models/en/thorsten/tacotron2-DDC")
# tts.tts_with_vc_to_file(
# """
# Bonjour à tous ! Bienvenue sur ma chaîne, je suis Louis en vlog, votre guide local, directement depuis la splendide Nouvelle-Calédonie. Ici, on explore, on découvre et on s'amuse sous le soleil du Pacifique ! Préparez-vous à plonger dans les eaux cristallines, à grimper les sommets verdoyants et à vivre des rencontres inoubliables avec notre incroyable communauté. Ensemble, nous allons voyager à travers ce territoire de légendes, et qui sait, peut-être même que nous dévoilerons quelques-uns de ses secrets... Alors, attachez bien votre ceinture, c'est parti pour une nouvelle aventure ! Et n'oubliez pas, si vous aimez ce que vous voyez, cliquez sur 'J'aime', partagez et abonnez-vous pour ne rien manquer. C'est parti !
# """,
#     speaker_wav="target-upgrade.wav",
#     file_path="output.wav"
# )

# from TTS.api import TTS
# api = TTS(model_name="tts_models/eng/fairseq/vits", gpu=True)
# api.tts_to_file("This is a test.", file_path="output.wav")

# TTS with on the fly voice conversion
api = TTS("tts_models/multilingual/multi-dataset/bark", gpu=True)
api.tts_with_vc_to_file(
"""
[laughs] Y'a pas sport demain,j'imagine ? [sighs]
""",
    speaker_wav="bark_voices/gael/speaker.wav",
    file_path="output.wav"
)
# play text in notebook
Audio("output.wav", rate=SAMPLE_RATE)

In [ ]:
from bark import SAMPLE_RATE, generate_audio, preload_models
from scipy.io.wavfile import write as write_wav
from IPython.display import Audio
preload_models()
# download and load all models
# preload_models()

# # generate audio from text
# text_prompt = """
#      Hello, my name is Suno. And, uh — and I like pizza. [laughs] 
#      But I also have other interests such as playing tic tac toe.
# """
# audio_array = generate_audio(text_prompt, history_prompt="bark_voices/louis")

# # save audio to disk
# write_wav("bark_generation.wav", SAMPLE_RATE, audio_array)
  
# # play text in notebook
# Audio(audio_array, rate=SAMPLE_RATE)

In [ ]:

# generate audio from text
text_prompt = """
[laughs] Y'a pas sport demain,j'imagine ? [sighs]
"""
audio_array = generate_audio(text_prompt, history_prompt="v2/fr_speaker_0")

# save audio to disk
write_wav("bark_generation.wav", SAMPLE_RATE, audio_array)
  
# play text in notebook
Audio(audio_array, rate=SAMPLE_RATE)

In [ ]:
%pip install git+https://github.com/suno-ai/bark.git

In [ ]:
from bark import SAMPLE_RATE, generate_audio, preload_models
from IPython.display import Audio

preload_models()

In [ ]:
text_prompt ='''
♪D'où viens-tu, petite tache noire ? 
Est-ce l'Éthiopie ou le Salvador ?
J'attends, tasse à la main,♪ 
Sers-moi encore. [soupir satisfait]
Café, où es-tu ? [ton désespéré]
Macchiato, où es-tu ? [en recherche]
Cappuccino, où es-tu ? [nostalgique]
Café, où es-tu ? Où es-tu ? Où es-tu ?♪ [urgence croissante]
'''
audio_array = generate_audio(text_prompt, history_prompt="v2/fr_speaker_3")
Audio(audio_array, rate=SAMPLE_RATE)

In [ ]:
text_prompt ='''
(Verse 1)
♪ D'où viens-tu, petite tache noire ? ♪
♪ Est-ce l'Éthiopie ou le Salvador ? ♪
♪ J'attends, tasse à la main, ♪
♪ Sers-moi encore. ♪          
'''
audio_array2 = generate_audio(text_prompt, history_prompt="v2/fr_speaker_0")
Audio(audio_array, rate=SAMPLE_RATE)

In [ ]:
%pip install git+https://github.com/huggingface/transformers.git

In [ ]:
from transformers import AutoProcessor, BarkModel

processor = AutoProcessor.from_pretrained("suno/bark")
model = BarkModel.from_pretrained("suno/bark")



In [ ]:

voice_preset = "v2/fr_speaker_6"
inputs = processor('''
(Verse 1)
♪ D'où viens-tu, petite tache noire ?
Est-ce l'Éthiopie ou le Salvador ?
J'attends, tasse à la main,
Sers-moi encore. ♪
              
''', voice_preset=voice_preset)

audio_array = model.generate(**inputs)
audio_array = audio_array.cpu().numpy().squeeze()



In [ ]:
from IPython.display import Audio

sample_rate = model.generation_config.sample_rate
Audio(audio_array, rate=sample_rate)

In [ ]:
import scipy

sample_rate = model.generation_config.sample_rate
scipy.io.wavfile.write("bark_out.wav", rate=sample_rate, data=audio_array)